In [6]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

def on_button_click():
    # Retrieve the paths from entry widgets
    path = path_entry.get()
    draft = draft_entry.get()
    pdf = pdf_entry.get()
    if path:
        try:
            # Import necessary libraries for data manipulation and PDF generation
            import pandas as pd
            import xlwings as xw
            from openpyxl import load_workbook
            import win32com.client
            import psutil
            import time
            import gc
            import os
            from urllib.parse import unquote

            # Load data from Excel files
            owner_name_table = pd.read_excel(path, skiprows=[0], sheet_name="Owner Name")
            sales_table = pd.read_excel(path, sheet_name="Sales")
            draft_table = pd.read_excel(draft, sheet_name="Sheet1")
            count = 0 # Initialize a counter for tracking generated PDFs

            # Extract entity names and states to lists
            entity_name_list = owner_name_table["Name of the Entity - customer"].tolist()
            state_list = owner_name_table["State"].tolist()

            # Initialize xlwings app and workbook for PDF generation
            app = xw.App(visible=False)
            wb = app.books.open(draft)
            
            # Iterate over each entity name for processing
            for i in range(len(entity_name_list)):
                final_name = owner_name_table[(owner_name_table["Name of the Entity - customer"] == entity_name_list[i]) & (owner_name_table["State"] == state_list[i])]
                final_name_list = str(final_name["File Name"][i])
                
                # Perform string manipulations for the filename
                final_name_list = final_name_list.replace(" ", "_")
                final_name_list = final_name_list.replace("_", "\u00A0")
                final_name_list = final_name_list.replace("/", "_")
                final_name_list = final_name_list.replace("`", "'")
                finalname = f"{i}.pdf"
                check_path = os.path.join(pdf, f"{final_name_list}.pdf")
                
                # Check if the PDF already exists
                if not os.path.exists(check_path):
                    count = count + 1 # Increment the counter if a new PDF is to be generated
                    # Filter the sales table for matching entity and state
                    draft = (sales_table["Name of the Entity - customer"] == entity_name_list[i]) & (sales_table["State"] == state_list[i])
                    sales_table_draft = sales_table[draft]
                    
                    # Calculate GST and plastic quantity totals
                    gst_paid_total = sales_table_draft["GST Paid"].sum()
                    plastic_quantity_total = sales_table_draft["Total Plastic Quantity (Metric ton)"].sum() / 1000000
                    plastic_quantity_total_tons = f"{plastic_quantity_total:.7f}"
                    plastic_quantity_total_tons
                    
                    # Extract invoice numbers to a list
                    invoice_numbers = sales_table_draft["Generated Invoice Number (Bot will paste here once generated from portal)"].tolist()

                    # Update the draft table with calculated values
                    for j in range(20):
                        if pd.isna(sales_table_draft.iloc[0, j]):
                            draft_table.iloc[0, j] = "NA"
                        else:
                            draft_table.iloc[0, j] = sales_table_draft.iloc[0, j]
                    draft_table["GST Paid"] = gst_paid_total
                    draft_table["Total Plastic Quantity (Metric ton)"] = plastic_quantity_total_tons
                    draft_table["Generated Invoice Number (Bot will paste here once generated from portal)"] = "\n".join(invoice_numbers)
                    draft_table_to_excel = draft_table
                    draft_table_to_excel

                    # Write the updated draft table to the Excel sheet
                    ws = wb.sheets["Sheet1"]
                    ws.range('A2').options(index=False, header=False).value = draft_table

                    # Export the sheet as a PDF to the specified folder
                    ws.api.ExportAsFixedFormat(0, f"{pdf}/{final_name_list}.pdf")
                else:
                    continue
        except Exception as e:
            messagebox.showerror("Error", "Error: {}".format(str(e)))
        finally:
            messagebox.showinfo("Done", f"Generated {count} pdf's") 
            wb.close()
            app.quit()

# Initialize the top-level widget as a pop-up window.            
top = tk.Toplevel()
app = top
app.title("XLSX to PDF conventer")
width = 550
height = 200

# Calculate and set the position for the application window to center it on the screen.
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
x_position = int((screen_width/2) - (width/2))
y_position = int((screen_height/2) - (height/2))
app.geometry(f"{width}x{height}+{x_position}+{y_position}")

#MAIN file selection section

path_frame = tk.Frame(app)
path_frame.pack(pady=10)

path_entry = tk.Entry(path_frame, width=60)
path_entry.pack(side=tk.LEFT, padx=5)

# Function to open a file dialog and select the main XLSX file
def browse_file():
    filepath = filedialog.askopenfilename()
    path_entry.delete(0, tk.END) # Clear the entry widget
    path_entry.insert(0, filepath) # Insert the selected file path into the entry widget

# Button to trigger the file selection dialog
browse_button = tk.Button(path_frame, text="Choose Main file", command=browse_file)
browse_button.pack(side=tk.LEFT, padx=5)

#DRAFT file selection section

draft_frame = tk.Frame(app)
draft_frame.pack(pady=10)

draft_entry = tk.Entry(draft_frame, width = 60)
draft_entry.pack(side=tk.LEFT, padx=5)

# Function to select the draft file for processing
def draft_function():
    draftpath = filedialog.askopenfilename()
    draft_entry.delete(0, tk.END)
    draft_entry.insert(0, draftpath)

# Button to trigger the draft file selection dialog
draft_button = tk.Button(draft_frame, text="Choose Draft file", command=draft_function)
draft_button.pack(side=tk.LEFT, padx=5)

# PDF's destination folder selection section

pdf_frame = tk.Frame(app)
pdf_frame.pack(pady=10)

pdf_entry = tk.Entry(pdf_frame, width = 52)
pdf_entry.pack(side=tk.LEFT, padx=5)

# Function to select the destination folder for the generated PDFs
def pdf_function():
    pdfpath = filedialog.askdirectory()
    pdf_entry.delete(0, tk.END)
    pdf_entry.insert(0, pdfpath)

# Button to trigger the destination folder selection dialog
pdf_button = tk.Button(pdf_frame, text="Choose destination folder", command=pdf_function)
pdf_button.pack(side=tk.LEFT, padx=5)

# Button to execute the conversion process

button = tk.Button(app, text="Run script", command=on_button_click)
button.pack(pady=10)

# Start the GUI event loop
app.mainloop()


C:\Users\30791552\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


ZETA ENTERPRISES Kerala FY 2022-2023
